In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K
from keras.optimizers import Adam
import os
from IPython.display import display
import json
from os.path import exists

%autosave 5

Autosaving every 5 seconds


In [2]:
def loadData(dataset,split=0):
    if(split == 0):
        print("In non split")
        return tf.keras.preprocessing.image_dataset_from_directory(f"../Data/{dataset}",
                                                                   labels='inferred',shuffle=True,seed=51)
    else:
        print("In split")
        return tf.keras.preprocessing.image_dataset_from_directory(f"../Data/{dataset}",
                                                                   labels='inferred',shuffle=True,seed=51,
                                                            validation_split=split,subset='both')

In [3]:
def f1_score(y_true, y_pred): #taken from old keras source code 
    y_true = K.round(K.clip(y_true, 0, 1))
    y_pred = K.round(K.clip(y_pred, 0, 1))
    
    true_positives = K.sum(y_true * y_pred)
    predicted_positives = K.sum(y_pred)
    possible_positives = K.sum(y_true)
    
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

In [4]:
def createModel(convFilters1, convFilters2, convFilters3, convFilters4, numberOfFCLayers, numberOfNeuronsPerFCLayer, adamLearningRate,L2Rate):
    model = keras.Sequential()
        
    #convPool1
    model.add(keras.layers.Conv2D(convFilters1,(3,3), activation='relu',padding='valid'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2,2), padding='valid'))
    #convPool2
    model.add(keras.layers.Conv2D(convFilters2,(3,3), activation='relu',padding='valid'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2,2), padding='valid'))
    #convPool3
    model.add(keras.layers.Conv2D(convFilters3,(3,3), activation='relu',padding='valid'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2,2), padding='valid'))
    #finalConv
    model.add(keras.layers.Conv2D(convFilters4,(3,3), activation='relu',padding='valid'))
    
    model.add(keras.layers.Flatten())
    
    for layer in range(numberOfFCLayers):
        if layer == numberOfFCLayers - 1:
            model.add(keras.layers.Dense(1,activation='sigmoid'))
        else:
            model.add(keras.layers.Dense(numberOfNeuronsPerFCLayer,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(L2Rate)))
            
    adamOptimizer = keras.optimizers.legacy.Adam(learning_rate=adamLearningRate)       
    model.compile(optimizer=adamOptimizer,loss='binary_crossentropy', metrics=f1_score)
    return model

In [5]:
def createModelParametersDF(n_convFilters1,n_convFilters2,n_convFilters3,n_convFilters4,
                            n_FCLayers,n_NeuronsPerFCLayers,n_Epochs,adamLearningRates,
                            L2Rates,trainScores,devScores):
    modelParameters = dict()
    modelParameters['n_convFilters1'] = n_convFilters1
    modelParameters['n_convFilters2'] = n_convFilters2
    modelParameters['n_convFilters3'] = n_convFilters3
    modelParameters['n_convFilters4'] = n_convFilters4
    modelParameters['n_FCLayers'] = n_FCLayers
    modelParameters['n_NeuronsPerFCLayers'] = n_NeuronsPerFCLayers
    modelParameters['n_Epochs'] = n_Epochs
    modelParameters['adamLearningRates'] = adamLearningRates
    modelParameters['L2Rates'] = L2Rates
    modelParameters['trainScore'] = trainScores
    modelParameters['devScore'] = devScores

    modelParametersDF = pd.DataFrame(modelParameters, columns=modelParameters.keys())
    return modelParametersDF

In [6]:
def createRangeFromMidpoint(midpoint,range,mandatoryMinimum=1):
    possibleMin = int(midpoint-(range/2))
    possibleMin = max([mandatoryMinimum,possibleMin])
    possibleMax = int(midpoint+(range/2))
    possibleRange = np.arange(possibleMin,possibleMax)
    return possibleRange

In [7]:
def generateAdamLearningRate(minVal=1e-4,maxVal=1e-2):
    minVal = np.log10(minVal)
    maxVal = np.log10(maxVal)
    learningRatePower = np.random.random() * (maxVal - minVal) + minVal
    learningRate = np.power(10,learningRatePower)
    return learningRate

In [8]:
def generateL2(minVal=1e-2,maxVal=1e3):
    minVal = np.log10(minVal)
    maxVal = np.log10(maxVal)
    l2Power = np.random.random() * (maxVal - minVal) + minVal
    l2 = np.power(10,l2Power)
    return l2

In [9]:
def calculateCriticalPoints(top5ParamList):
    lowPoint = np.min(top5ParamList)
    highPoint = np.max(top5ParamList)
    return (lowPoint,highPoint)

In [10]:
def calculateLogisticCriticalPoints(top5ParamList):
    top5Log10ParamList = np.log10(top5ParamList)
    log10criticalPointTuple = calculateCriticalPoints(top5Log10ParamList)
    criticalPointTuple = (np.power(10,log10criticalPointTuple[0]),np.power(10,log10criticalPointTuple[1]))
    return criticalPointTuple

In [11]:
def getAdjustedRange(top5ParamList):
    lowerValue = int(np.max([1,np.min(top5ParamList)]))
    upperValue = int(np.max(top5ParamList))
    
    if lowerValue == upperValue:
        return createRangeFromMidpoint(lowerValue,2*lowerValue)
    return np.arange(lowerValue,upperValue)

In [12]:
def displayFinalResults(parameterFileName):
    if (exists(parameterFileName)):
        with open(parameterFileName) as d:
            finalResults = json.load(d)
            resultsDictionary = dict()
            for key in finalResults.keys():
                resultsDictionary[key] = [finalResults[key]]
            resultsDF = pd.DataFrame(resultsDictionary,columns = finalResults.keys())
            print('Final Model')
            display(resultsDF)

In [13]:
def main():

    possibleConvFilters1 = createRangeFromMidpoint(32,64)
    possibleConvFilters2 = createRangeFromMidpoint(32,64)
    possibleConvFilters3 = createRangeFromMidpoint(32,64)
    possibleConvFilters4 = createRangeFromMidpoint(32,64)

    possibleNumberOfFCLayers = createRangeFromMidpoint(10,20)
    possibleNumberOfNeuronsPerFCLayer = createRangeFromMidpoint(10,20)

    possibleNumberOfEpochs = createRangeFromMidpoint(10,20)
    dropoutCriticalPoints = (0,1)
    adamLearningRateCriticalPoints = (1e-4,1e-2)
    L2CriticalPoints = (1e-2,1e3)    
    
    trial = 0
    bestDevScore = 0
    
    train = loadData("training")
    dev,test = loadData("testing",.5)

    choose = np.random.choice

    n_convFilters1 = []
    n_convFilters2 = []
    n_convFilters3 = []
    n_convFilters4 = []
    n_FCLayers = []
    n_NeuronsPerFCLayers = []
    n_Epochs = []
    adamLearningRates = []
    L2Rates = []
    trainScores = []
    devScores = []
    
    while trial < 100:
        convFilters1 = choose(possibleConvFilters1)
        convFilters2 = choose(possibleConvFilters2)
        convFilters3 = choose(possibleConvFilters3)
        convFilters4 = choose(possibleConvFilters4)

        numberOfFCLayers = choose(possibleNumberOfFCLayers)
        numberOfNeuronsPerFCLayer = choose(possibleNumberOfNeuronsPerFCLayer)

        numberOfEpochs = choose(possibleNumberOfEpochs)
        
        adamLearningRate = generateAdamLearningRate(adamLearningRateCriticalPoints[0],adamLearningRateCriticalPoints[1])
        L2Rate = generateL2(L2CriticalPoints[0],L2CriticalPoints[1])
        

        model = createModel(convFilters1, convFilters2, convFilters3, convFilters4,
                            numberOfFCLayers, numberOfNeuronsPerFCLayer, adamLearningRate,L2Rate)
    
        model.fit(train,epochs=numberOfEpochs,verbose=0)

        model_path = f'../Models/FlipDetectionModelTrials/fd_model_{trial}.h5'
        model.save(model_path)
        model_size = os.path.getsize(model_path) / (1024 * 1024)
        if model_size < 40:
            print()
            print('trainScore')
            trainScore = model.evaluate(train)[1]
            print('devScore')
            devScore = model.evaluate(dev)[1]

            if (devScore > 0.91) and (devScore > bestDevScore):
                testScore = model.evaluate(test)[1]
                model_path = f'../Models/best_fd_model_.h5'
                model.save(model_path)
                bestModelParams = {
                    'n_convFilters1' : int(convFilters1),
                    'n_convFilters2' : int(convFilters2),
                    'n_convFilters3' : int(convFilters3),
                    'n_convFilters4' : int(convFilters4),
                    'n_FCLayers' : int(numberOfFCLayers),
                    'n_NeuronsPerFCLayers' : int(numberOfNeuronsPerFCLayer),
                    'n_Epochs' : int(numberOfEpochs),
                    'adamLearningRates' : adamLearningRate,
                    'L2Rates' : L2Rate,
                    'modelSize' : model_size,
                    'trainScore': trainScore,
                    'devScore': devScore,
                    'testScore': testScore
                }
                with open('../Models/best_fd_model_params.json', 'w') as f:
                    json.dump(bestModelParams, f)
                bestDevScore = devScore       

            n_convFilters1.append(convFilters1)
            n_convFilters2.append(convFilters2)
            n_convFilters3.append(convFilters3)
            n_convFilters4.append(convFilters4)
            
            n_FCLayers.append(numberOfFCLayers)
            n_NeuronsPerFCLayers.append(numberOfNeuronsPerFCLayer)
            n_Epochs.append(numberOfEpochs)
            
            adamLearningRates.append(adamLearningRate)
            L2Rates.append(L2Rate)
            trainScores.append(trainScore)
            devScores.append(devScore)
            
            print('concluding trial ',trial)
            trial += 1
        else:
            print(f'redoing trial {trial}. Model was {model_size}MB.')
            failedTrial = createModelParametersDF([convFilters1],[convFilters2],[convFilters3],[convFilters4],
                                                  [numberOfFCLayers],[numberOfNeuronsPerFCLayer],[numberOfEpochs],
                                                  [adamLearningRate],[L2Rate],[np.nan],[np.nan])
            display(failedTrial)
        
            
        if (trial % 10 == 9): 
            modelParametersDF = createModelParametersDF(n_convFilters1,n_convFilters2,n_convFilters3,n_convFilters4,
                                                        n_FCLayers,n_NeuronsPerFCLayers,n_Epochs,adamLearningRates,L2Rates,
                                                        trainScores,devScores)
            modelParametersDF = modelParametersDF.sort_values(by='trainScore', ascending=False)
            display(modelParametersDF)
            
            top5 = modelParametersDF[0:5]
            possibleConvFilters1 = getAdjustedRange(top5['n_convFilters1'])
            possibleConvFilters2 = getAdjustedRange(top5['n_convFilters2'])
            possibleConvFilters3 = getAdjustedRange(top5['n_convFilters3'])
            possibleConvFilters4 = getAdjustedRange(top5['n_convFilters4'])
        
            possibleNumberOfFCLayers = getAdjustedRange(top5['n_FCLayers'])
            possibleNumberOfNeuronsPerFCLayer = getAdjustedRange(top5['n_NeuronsPerFCLayers'])
        
            possibleNumberOfEpochs = getAdjustedRange(top5['n_Epochs'])
            adamLearningRateCriticalPoints = calculateLogisticCriticalPoints(top5['adamLearningRates'])
            L2CriticalPoints = calculateLogisticCriticalPoints(top5['L2Rates'])

            n_convFilters1 = []
            n_convFilters2 = []
            n_convFilters3 = []
            n_convFilters4 = []
            n_FCLayers = []
            n_NeuronsPerFCLayers = []
            n_Epochs = []
            adamLearningRates = []
            L2Rates = []
            trainScores = []
            devScores = []

            if bestDevScore > 0.91:
                trial = 101
                
    displayFinalResults('../Models/best_fd_model_params.json')
            
    

In [14]:
main()

In non split
Found 2392 files belonging to 2 classes.
In split
Found 597 files belonging to 2 classes.
Using 299 files for training.
Using 298 files for validation.


2024-09-22 10:11:33.461980: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-09-22 10:11:33.462133: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-09-22 10:11:33.740082: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz



trainScore
75/75 [==============================] - 10s 122ms/step - loss: 0.7315 - f1_score: 0.8808
devScore


2024-09-22 10:15:36.988125: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [299]
	 [[{{node Placeholder/_4}}]]
2024-09-22 10:15:36.988288: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [299]
	 [[{{node Placeholder/_0}}]]


10/10 [==============================] - 1s 79ms/step - loss: 0.7410 - f1_score: 0.8837
concluding trial  0

trainScore
75/75 [==============================] - 13s 168ms/step - loss: 0.6928 - f1_score: 0.0000e+00
devScore
10/10 [==============================] - 2s 123ms/step - loss: 0.6915 - f1_score: 0.0000e+00
concluding trial  1

trainScore
75/75 [==============================] - 13s 170ms/step - loss: 1.7363 - f1_score: 0.4390
devScore
10/10 [==============================] - 2s 123ms/step - loss: 1.7427 - f1_score: 0.4584
concluding trial  2

trainScore
75/75 [==============================] - 9s 117ms/step - loss: 0.8418 - f1_score: 0.0000e+00
devScore
10/10 [==============================] - 1s 76ms/step - loss: 0.8405 - f1_score: 0.0000e+00
concluding trial  3

trainScore
75/75 [==============================] - 18s 230ms/step - loss: 0.6927 - f1_score: 0.0000e+00
devScore
10/10 [==============================] - 2s 181ms/step - loss: 0.6917 - f1_score: 0.0000e+00
concluding

...

2024-09-22 10:58:44.059036: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [298]
	 [[{{node Placeholder/_4}}]]
2024-09-22 10:58:44.059153: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [298]
	 [[{{node Placeholder/_4}}]]


10/10 [==============================] - 1s 133ms/step - loss: 0.4580 - f1_score: 0.9499
concluding trial  10

trainScore
75/75 [==============================] - 12s 146ms/step - loss: 19.4037 - f1_score: 0.0000e+00
devScore
10/10 [==============================] - 1s 102ms/step - loss: 19.4028 - f1_score: 0.0000e+00
concluding trial  11

trainScore
75/75 [==============================] - 8s 102ms/step - loss: 689.8909 - f1_score: 0.0000e+00
devScore
10/10 [==============================] - 1s 59ms/step - loss: 689.8901 - f1_score: 0.0000e+00
concluding trial  12

trainScore
75/75 [==============================] - 12s 155ms/step - loss: 27.8479 - f1_score: 0.0000e+00
devScore
10/10 [==============================] - 2s 109ms/step - loss: 27.8469 - f1_score: 0.0000e+00
concluding trial  13

trainScore
75/75 [==============================] - 8s 95ms/step - loss: 38.7270 - f1_score: 0.0000e+00
devScore
10/10 [==============================] - 1s 57ms/step - loss: 38.7261 - f1_score: 0

,n_convFilters1,n_convFilters2,n_convFilters3,n_convFilters4,n_FCLayers,n_NeuronsPerFCLayers,n_Epochs,adamLearningRates,L2Rates,trainScore,devScore
1,30,35,53,14,3,11,17,0.000434,0.596852,0.974815,0.936822
0,16,4,23,9,17,6,9,0.003128,0.555927,0.000000,0.000000
2,45,13,25,15,16,2,6,0.002014,10.533279,0.000000,0.000000
3,11,19,45,44,12,2,16,0.000325,123.529028,0.000000,0.000000
4,41,26,19,10,15,9,6,0.001886,82.888678,0.000000,0.000000
5,16,25,45,39,16,1,5,0.002422,12.686227,0.000000,0.000000
6,14,26,57,36,10,8,8,0.005658,0.511021,0.000000,0.000000
7,37,15,19,31,12,7,5,0.000805,10.180509,0.000000,0.000000
8,15,46,33,44,15,9,13,0.000651,59.466246,0.000000,0.000000
9,35,34,41,10,12,7,11,0.003633,120.847365,0.000000,0.000000


Final Model


,n_convFilters1,n_convFilters2,n_convFilters3,n_convFilters4,n_FCLayers,n_NeuronsPerFCLayers,n_Epochs,adamLearningRates,L2Rates,modelSize,trainScore,devScore,testScore
0,30,35,53,14,3,11,17,0.000434,0.596852,1.835278,0.974815,0.936822,0.949896
